In [2]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [3]:
import csv, datetime, fcntl, glob, json, math, os, re, subprocess, sys, time, urllib2, xml.dom.minidom, pytz
from dateutil import rrule
from dateutil import tz
import dateutil.parser
import numpy as np

# To install dateutil on Ubuntu
# sudo pip install python-dateutil

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('python-utils/esdr-library.ipynb')

In [4]:
esdr = Esdr(None)
product = esdr.get_product_by_name('PurpleAir')
product

{u'created': u'2017-11-20T15:17:13.000Z',
 u'creatorUserId': 3,
 u'defaultChannelSpecs': {},
 u'description': u'PurpleAir',
 u'id': 69,
 u'modified': u'2017-11-20T15:17:13.000Z',
 u'name': u'PurpleAir',
 u'prettyName': u'PurpleAir',
 u'vendor': u'PurpleAir'}

In [5]:
feeds = []
# Approx bounding box for Allegheny County
region = 'latitude>40.192204,latitude<40.674084,longitude<-79.688618,longitude>-80.361022'
offset = 0
while True:
    response = esdr.api('GET', '/api/v1/feeds', {'whereAnd' : 'productId=%d,%s' % (product['id'], region), 'offset': str(offset)})
    data = response['data']
    totalCount = data['totalCount']
    nrecs = len(data['rows'])
    feeds.extend(data['rows'])
    limit = data['limit']
    total_recvd = offset + nrecs
    #print(offset, nrecs, limit, totalCount)
    if total_recvd >= totalCount:
        break
    offset += nrecs

print 'Retrieved %d feeds' % len(feeds)
print 'First feed: %s' % feeds[0]

Retrieved 183 feeds
First feed: {u'apiKeyReadOnly': u'2bd4d40a3059b5ef56a3f6a6b78386569da2c2580fcd5c42e497072119f7b79c', u'minTimeSecs': 1522247982.349, u'name': u'Monaca,PA PurpleAir', u'created': u'2018-03-28T14:51:40.000Z', u'isPublic': 1, u'userId': 3, u'modified': u'2018-12-23T20:27:30.000Z', u'longitude': -80.311057, u'channelBounds': {u'channels': {u'PM2_5': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 3810.59, u'minValue': 0}, u'stats_v': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 970.64, u'minValue': 0}, u'stats_pm': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 970.64, u'minValue': 0}, u'temp_f': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 105, u'minValue': 27}, u'humidity': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 84, u'minValue': 11}, u'pressure': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982

In [6]:
def sample_feed(feed, interval, from_epoch, to_epoch):
    feed_id = feed['id']
    export_url = 'https://esdr.cmucreatelab.org/api/v1/feeds/%d/channels/PM2_5/export?from=%d&to=%d&format=json' % (feed_id, from_epoch, to_epoch)
    data = json.load(urllib2.urlopen(export_url))['data']
    
    
    print('Feed %d has %d samples' % (feed_id, len(data)))
    if len(data) == 0:
        print('No samples, skipping')
        return None

    
    dat = []
    start_time = from_epoch
    end_time = from_epoch + interval - 1

    prevSample = 0
    while((end_time+1) < to_epoch):
        curbin = []
        for sample in data:
            if(sample[0] >= start_time and sample[0] <= end_time):
                curbin.append(sample[1])
        newSample = prevSample
        if(len(curbin) > 0):    
            newSample =  sum(curbin)/len(curbin)
        if(newSample == 0):
            newSample = prevSample
     
        start_time += interval
        end_time += (interval-1)
        
        dat.append(newSample)
        if(newSample != 0):
            prevSample = newSample
    
    
        
    row = ['feed_%d' % feed_id, feed['latitude'], feed['longitude']] + list(dat)
 
    return row

    
    

In [11]:
sensor =  sample_feed(feeds[110], 300, 1532822400, 1532822400+86400)
sensor.pop(0)
print sensor
print len(np.array(sensor).astype(np.float32).tobytes())

Feed 14321 has 0 samples
No samples, skipping


AttributeError: 'NoneType' object has no attribute 'pop'

In [13]:
def output_raw_binary(cur_date):
    
    #Gets datetime from string
    date_datetime = datetime.datetime.strptime(cur_date, "%Y-%m-%d")
    #Convert to datetime
    epoch_time = (pytz.timezone('America/New_York').localize(date_datetime) - datetime.datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds()
    raw_data = []
    bin_data = []
    count = 0
    for feed in feeds:

        #Adds 24 hours to starting epoch_time
        #Get each sensor feed sampled at 5 mins
        sensor = sample_feed(feed, 300, epoch_time, epoch_time+86400)
        
        #Adds all non-NULL sensors to raw/bin data
        if(sensor is not None and len(sensor) > 0):
            count += 1
            raw_data.append(sensor)
            sensor.pop(0)
            bin_data.append(sensor)
    
    #Convert to binary
    raw_binary = np.array(bin_data).astype(np.float32).tobytes()

    #Outputs binary file
    path = "aggregates/"
    bin_outfile = open(path + 'purpleair_'  + cur_date + 'tmp.bin', 'w+')
    bin_outfile.write(raw_binary)
    bin_outfile.close()

    #Outputs raw json file
    path = "aggregates/"
    with open(path + 'purpleair_'  + cur_date + 'tmp.json', 'w+') as outfile:  
        json.dump(raw_data, outfile)

    os.rename(path + 'purpleair_'  + cur_date + 'tmp.json', path + 'purpleair_'  + cur_date + '.json')
    os.rename(path + 'purpleair_'  + cur_date + 'tmp.bin',  path + 'purpleair_'  + cur_date + '.bin')


In [14]:
#Get current time
curDate = datetime.datetime.now().strftime("%Y-%m-%d")
print(curDate)
output_raw_binary(curDate)


for i in range(0, 16):
    curDate = (datetime.datetime.now()-datetime.timedelta(days=i)).strftime("%Y-%m-%d")
    print(curDate)
    output_raw_binary(curDate)
    


2019-01-30
Feed 17199 has 0 samples
No samples, skipping
Feed 17200 has 0 samples
No samples, skipping
Feed 25600 has 0 samples
No samples, skipping
Feed 15876 has 0 samples
No samples, skipping
Feed 15877 has 0 samples
No samples, skipping
Feed 25601 has 0 samples
No samples, skipping
Feed 14945 has 0 samples
No samples, skipping
Feed 14946 has 0 samples
No samples, skipping
Feed 20581 has 0 samples
No samples, skipping
Feed 20582 has 0 samples
No samples, skipping
Feed 19958 has 0 samples
No samples, skipping
Feed 19959 has 0 samples
No samples, skipping
Feed 25765 has 0 samples
No samples, skipping
Feed 25766 has 0 samples
No samples, skipping
Feed 17964 has 0 samples
No samples, skipping
Feed 17965 has 0 samples
No samples, skipping
Feed 19703 has 0 samples
No samples, skipping
Feed 19704 has 0 samples
No samples, skipping
Feed 18986 has 0 samples
No samples, skipping
Feed 18987 has 0 samples
No samples, skipping
Feed 19028 has 0 samples
No samples, skipping
Feed 19029 has 0 sample

KeyboardInterrupt: 

In [15]:


curDate = '2018-10-01'
while(curDate != '2019-01-30'):
    print(curDate)
    output_raw_binary(curDate)
    curDate = (datetime.datetime.strptime(curDate, '%Y-%m-%d')+datetime.timedelta(days=1)).strftime("%Y-%m-%d")

2018-10-01
Feed 17199 has 5 samples
Feed 17200 has 5 samples
Feed 25600 has 0 samples
No samples, skipping
Feed 15876 has 10 samples
Feed 15877 has 10 samples
Feed 25601 has 0 samples
No samples, skipping
Feed 14945 has 10 samples
Feed 14946 has 10 samples
Feed 20581 has 9 samples
Feed 20582 has 9 samples
Feed 19958 has 0 samples
No samples, skipping
Feed 19959 has 0 samples
No samples, skipping
Feed 25765 has 0 samples
No samples, skipping
Feed 25766 has 0 samples
No samples, skipping
Feed 17964 has 8 samples
Feed 17965 has 8 samples
Feed 19703 has 3 samples
Feed 19704 has 3 samples
Feed 18986 has 10 samples
Feed 18987 has 10 samples
Feed 19028 has 10 samples
Feed 19029 has 10 samples
Feed 14927 has 6 samples
Feed 14928 has 6 samples
Feed 19956 has 9 samples
Feed 19957 has 9 samples
Feed 19302 has 6 samples
Feed 19303 has 6 samples
Feed 18679 has 6 samples
Feed 18680 has 6 samples
Feed 18106 has 10 samples
Feed 18107 has 10 samples
Feed 22641 has 0 samples
No samples, skipping
Feed 22

Feed 14761 has 0 samples
No samples, skipping
Feed 14762 has 0 samples
No samples, skipping
Feed 14763 has 0 samples
No samples, skipping
Feed 14764 has 0 samples
No samples, skipping
Feed 14768 has 7 samples
Feed 14769 has 7 samples
Feed 16691 has 0 samples
No samples, skipping
Feed 16692 has 0 samples
No samples, skipping
Feed 16693 has 0 samples
No samples, skipping
Feed 16694 has 0 samples
No samples, skipping
Feed 16695 has 0 samples
No samples, skipping
Feed 16696 has 0 samples
No samples, skipping
Feed 16697 has 0 samples
No samples, skipping
Feed 15499 has 0 samples
No samples, skipping
Feed 15500 has 0 samples
No samples, skipping
Feed 18084 has 1 samples
Feed 18085 has 1 samples
Feed 19895 has 0 samples
No samples, skipping
Feed 19896 has 0 samples
No samples, skipping
Feed 19741 has 0 samples
No samples, skipping
Feed 19742 has 0 samples
No samples, skipping
Feed 12956 has 0 samples
No samples, skipping
Feed 12957 has 0 samples
No samples, skipping
Feed 18702 has 10 samples


Feed 12948 has 0 samples
No samples, skipping
Feed 12949 has 0 samples
No samples, skipping
Feed 12950 has 0 samples
No samples, skipping
Feed 12951 has 0 samples
No samples, skipping
Feed 12942 has 0 samples
No samples, skipping
Feed 12943 has 0 samples
No samples, skipping
Feed 12946 has 0 samples
No samples, skipping
Feed 12947 has 0 samples
No samples, skipping
Feed 12958 has 0 samples
No samples, skipping
Feed 12959 has 0 samples
No samples, skipping
Feed 12954 has 0 samples
No samples, skipping
Feed 12955 has 0 samples
No samples, skipping
Feed 17888 has 0 samples
No samples, skipping
Feed 17889 has 0 samples
No samples, skipping
Feed 24309 has 0 samples
No samples, skipping
Feed 24310 has 0 samples
No samples, skipping
Feed 17783 has 0 samples
No samples, skipping
Feed 17784 has 0 samples
No samples, skipping
Feed 17195 has 0 samples
No samples, skipping
Feed 17196 has 0 samples
No samples, skipping
Feed 18827 has 0 samples
No samples, skipping
Feed 18828 has 0 samples
No sample

Feed 17889 has 0 samples
No samples, skipping
Feed 24309 has 0 samples
No samples, skipping
Feed 24310 has 0 samples
No samples, skipping
Feed 17783 has 0 samples
No samples, skipping
Feed 17784 has 0 samples
No samples, skipping
Feed 17195 has 0 samples
No samples, skipping
Feed 17196 has 0 samples
No samples, skipping
Feed 18827 has 0 samples
No samples, skipping
Feed 18828 has 0 samples
No samples, skipping
Feed 14314 has 0 samples
No samples, skipping
Feed 14315 has 0 samples
No samples, skipping
Feed 14316 has 0 samples
No samples, skipping
Feed 14317 has 0 samples
No samples, skipping
Feed 14318 has 0 samples
No samples, skipping
Feed 14319 has 0 samples
No samples, skipping
Feed 14320 has 0 samples
No samples, skipping
Feed 14321 has 0 samples
No samples, skipping
Feed 14322 has 0 samples
No samples, skipping
Feed 14323 has 0 samples
No samples, skipping
Feed 25790 has 0 samples
No samples, skipping
Feed 19846 has 0 samples
No samples, skipping
Feed 19847 has 0 samples
No sample

Feed 14323 has 0 samples
No samples, skipping
Feed 25790 has 0 samples
No samples, skipping
Feed 19846 has 0 samples
No samples, skipping
Feed 19847 has 0 samples
No samples, skipping
Feed 17892 has 0 samples
No samples, skipping
Feed 17893 has 0 samples
No samples, skipping
Feed 13902 has 0 samples
No samples, skipping
Feed 13903 has 0 samples
No samples, skipping
Feed 17978 has 0 samples
No samples, skipping
Feed 17979 has 0 samples
No samples, skipping
Feed 18412 has 0 samples
No samples, skipping
Feed 18413 has 0 samples
No samples, skipping
Feed 17966 has 0 samples
No samples, skipping
Feed 17967 has 0 samples
No samples, skipping
Feed 25789 has 0 samples
No samples, skipping
Feed 17970 has 0 samples
No samples, skipping
Feed 17971 has 0 samples
No samples, skipping
Feed 18153 has 0 samples
No samples, skipping
Feed 18154 has 0 samples
No samples, skipping
Feed 17983 has 0 samples
No samples, skipping
Feed 17984 has 0 samples
No samples, skipping
Feed 18096 has 0 samples
No sample

...

Feed 17967 has 0 samples
No samples, skipping
Feed 25789 has 0 samples
No samples, skipping
Feed 17970 has 0 samples
No samples, skipping
Feed 17971 has 0 samples
No samples, skipping
Feed 18153 has 0 samples
No samples, skipping
Feed 18154 has 0 samples
No samples, skipping
Feed 17983 has 0 samples
No samples, skipping
Feed 17984 has 0 samples
No samples, skipping
Feed 18096 has 0 samples
No samples, skipping
Feed 18097 has 0 samples
No samples, skipping
Feed 17785 has 0 samples
No samples, skipping
Feed 17786 has 0 samples
No samples, skipping
Feed 18271 has 0 samples
No samples, skipping
Feed 18272 has 0 samples
No samples, skipping
Feed 18269 has 0 samples
No samples, skipping
Feed 18270 has 0 samples
No samples, skipping
Feed 18210 has 0 samples
No samples, skipping
Feed 18211 has 0 samples
No samples, skipping
Feed 22300 has 0 samples
No samples, skipping
Feed 22301 has 0 samples
No samples, skipping
Feed 12698 has 751 samples
Feed 12699 has 747 samples
Feed 19988 has 0 samples
N

Feed 18412 has 0 samples
No samples, skipping
Feed 18413 has 0 samples
No samples, skipping
Feed 17966 has 0 samples
No samples, skipping
Feed 17967 has 0 samples
No samples, skipping
Feed 25789 has 0 samples
No samples, skipping
Feed 17970 has 0 samples
No samples, skipping
Feed 17971 has 0 samples
No samples, skipping
Feed 18153 has 0 samples
No samples, skipping
Feed 18154 has 0 samples
No samples, skipping
Feed 17983 has 0 samples
No samples, skipping
Feed 17984 has 0 samples
No samples, skipping
Feed 18096 has 0 samples
No samples, skipping
Feed 18097 has 0 samples
No samples, skipping
Feed 17785 has 0 samples
No samples, skipping
Feed 17786 has 0 samples
No samples, skipping
Feed 18271 has 0 samples
No samples, skipping
Feed 18272 has 0 samples
No samples, skipping
Feed 18269 has 0 samples
No samples, skipping
Feed 18270 has 0 samples
No samples, skipping
Feed 18210 has 0 samples
No samples, skipping
Feed 18211 has 0 samples
No samples, skipping
Feed 22300 has 0 samples
No sample

Feed 18413 has 0 samples
No samples, skipping
Feed 17966 has 0 samples
No samples, skipping
Feed 17967 has 0 samples
No samples, skipping
Feed 25789 has 0 samples
No samples, skipping
Feed 17970 has 0 samples
No samples, skipping
Feed 17971 has 0 samples
No samples, skipping
Feed 18153 has 0 samples
No samples, skipping
Feed 18154 has 0 samples
No samples, skipping
Feed 17983 has 0 samples
No samples, skipping
Feed 17984 has 0 samples
No samples, skipping
Feed 18096 has 0 samples
No samples, skipping
Feed 18097 has 0 samples
No samples, skipping
Feed 17785 has 0 samples
No samples, skipping
Feed 17786 has 0 samples
No samples, skipping
Feed 18271 has 0 samples
No samples, skipping
Feed 18272 has 0 samples
No samples, skipping
Feed 18269 has 0 samples
No samples, skipping
Feed 18270 has 0 samples
No samples, skipping
Feed 18210 has 0 samples
No samples, skipping
Feed 18211 has 0 samples
No samples, skipping
Feed 22300 has 0 samples
No samples, skipping
Feed 22301 has 0 samples
No sample

Feed 17979 has 0 samples
No samples, skipping
Feed 18412 has 0 samples
No samples, skipping
Feed 18413 has 0 samples
No samples, skipping
Feed 17966 has 0 samples
No samples, skipping
Feed 17967 has 0 samples
No samples, skipping
Feed 25789 has 0 samples
No samples, skipping
Feed 17970 has 0 samples
No samples, skipping
Feed 17971 has 0 samples
No samples, skipping
Feed 18153 has 0 samples
No samples, skipping
Feed 18154 has 0 samples
No samples, skipping
Feed 17983 has 0 samples
No samples, skipping
Feed 17984 has 0 samples
No samples, skipping
Feed 18096 has 0 samples
No samples, skipping
Feed 18097 has 0 samples
No samples, skipping
Feed 17785 has 0 samples
No samples, skipping
Feed 17786 has 0 samples
No samples, skipping
Feed 18271 has 0 samples
No samples, skipping
Feed 18272 has 0 samples
No samples, skipping
Feed 18269 has 0 samples
No samples, skipping
Feed 18270 has 0 samples
No samples, skipping
Feed 18210 has 0 samples
No samples, skipping
Feed 18211 has 0 samples
No sample

Feed 17979 has 0 samples
No samples, skipping
Feed 18412 has 0 samples
No samples, skipping
Feed 18413 has 0 samples
No samples, skipping
Feed 17966 has 0 samples
No samples, skipping
Feed 17967 has 0 samples
No samples, skipping
Feed 25789 has 0 samples
No samples, skipping
Feed 17970 has 0 samples
No samples, skipping
Feed 17971 has 0 samples
No samples, skipping
Feed 18153 has 0 samples
No samples, skipping
Feed 18154 has 0 samples
No samples, skipping
Feed 17983 has 0 samples
No samples, skipping
Feed 17984 has 0 samples
No samples, skipping
Feed 18096 has 0 samples
No samples, skipping
Feed 18097 has 0 samples
No samples, skipping
Feed 17785 has 0 samples
No samples, skipping
Feed 17786 has 0 samples
No samples, skipping
Feed 18271 has 0 samples
No samples, skipping
Feed 18272 has 0 samples
No samples, skipping
Feed 18269 has 0 samples
No samples, skipping
Feed 18270 has 0 samples
No samples, skipping
Feed 18210 has 0 samples
No samples, skipping
Feed 18211 has 0 samples
No sample